## Plot cancer type prediction results

In [1]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import mpmp.config as cfg
import mpmp.utilities.analysis_utilities as au

In [2]:
results_dir = Path(cfg.results_dir, 'cancer_type').resolve()

In [4]:
results_df = au.load_cancer_type_prediction_results(results_dir, 'cancer_type')
print(results_df.shape)
results_df.head()

(1584, 9)


,auroc,aupr,cancer_type,training_data,signal,seed,data_type,fold,experiment
0,0.98013,0.69226,HNSC,methylation,signal,42,train,0,cancer_type
1,0.97116,0.56608,HNSC,methylation,signal,42,test,0,cancer_type
2,0.96767,0.49104,HNSC,methylation,signal,42,cv,0,cancer_type
3,0.97838,0.63986,HNSC,methylation,signal,42,train,1,cancer_type
4,0.98098,0.73787,HNSC,methylation,signal,42,test,1,cancer_type
